In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [46]:
start_date="2018-01-01"
end_date="2023-12-31"
tickers=["AAPL","^GSPC","MSFT"]
data1=yf.download('AAPL',start=start_date,end=end_date)
data2=yf.download('^GSPC',start=start_date,end=end_date)
data3=yf.download('MSFT',start=start_date,end=end_date)

#for i in range(len(tickers)):
#    df=pd.concat([df,yf.download(tickers[i],start=start_date,end=end_date)],ignore_index=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [47]:
data1.rename(columns={'Open':'AAPL_Open'},inplace=True)
data1.rename(columns={'High':'AAPL_High'},inplace=True)
data1.rename(columns={'Low':'AAPL_Low'},inplace=True)
data1.rename(columns={'Close':'AAPL_Close'},inplace=True)
data1.rename(columns={'Adj Close':'AAPL_Adj Close'},inplace=True)
data1.rename(columns={'Volume':'AAPL_Volume'},inplace=True)

data2.rename(columns={'Open':'^GSPC_Open'},inplace=True)
data2.rename(columns={'High':'^GSPC_High'},inplace=True)
data2.rename(columns={'Low':'^GSPC_Low'},inplace=True)
data2.rename(columns={'Close':'^GSPC_Close'},inplace=True)
data2.rename(columns={'Adj Close':'^GSPC_Adj Close'},inplace=True)
data2.rename(columns={'Volume':'^GSPC_Volume'},inplace=True)

data3.rename(columns={'Open':'MSFT_Open'},inplace=True)
data3.rename(columns={'High':'MSFT_High'},inplace=True)
data3.rename(columns={'Low':'MSFT_Low'},inplace=True)
data3.rename(columns={'Close':'MSFT_Close'},inplace=True)
data3.rename(columns={'Adj Close':'MSFT_Adj Close'},inplace=True)
data3.rename(columns={'Volume':'MSFT_Volume'},inplace=True)


In [67]:
data=pd.merge(data1,data2,on='Date',how='right')
data=pd.merge(data,data3,on='Date',how='right')
data.to_csv('AAPL,MSFT,SPX.csv')
data_corection=torch.as_tensor(data.values,dtype=torch.float32)
data_corection=data_corection.to(device)
data_corection=(data_corection-data_corection.min())/(data_corection.max()-data_corection.min())


data_row=data_corection.t()
data_row.to(device)
data_row.size()
data_corection.size()
data_row

tensor([[8.4845e-10, 9.0784e-10, 9.0809e-10,  ..., 1.5879e-08, 1.6044e-08,
         1.6020e-08],
        [9.0208e-10, 9.5846e-10, 9.3140e-10,  ..., 1.5980e-08, 1.6096e-08,
         1.6070e-08],
        [8.2590e-10, 8.9356e-10, 8.9656e-10,  ..., 1.5738e-08, 1.5947e-08,
         1.5803e-08],
        ...,
        [5.1997e-09, 5.2398e-09, 5.3159e-09,  ..., 3.4079e-08, 3.4201e-08,
         3.4277e-08],
        [4.6114e-09, 4.6487e-09, 4.7197e-09,  ..., 3.4010e-08, 3.4131e-08,
         3.4207e-08],
        [2.2537e-03, 2.6123e-03, 2.1964e-03,  ..., 1.4940e-03, 1.4361e-03,
         1.8767e-03]])

In [74]:
class AI(nn.Module):
    def __init__(self,nUnits,nLayers):
        super(AI,self).__init__()
        
        # create dictionary to store the layers
        self.layers = nn.ModuleDict()
        self.nLayers = nLayers

        ### input layer
        self.layers['input'] = nn.Linear(18,1509)
        self.layers['hidden0']=nn.Linear(1509,nUnits)
        
        ### hidden layers
        for i in range(nLayers):
            self.layers[f'hidden{i+1}'] = nn.Linear(nUnits,nUnits)

        ### output layer
        self.layers['output'] = nn.Linear(nUnits,1)
    def forward(self,x):
        x=F.relu(self.layers['input'](x))
        for i in range(self.nLayers):
            x=F.relu(self.layers[f'hidden{i}'](x))
        x=self.layers['output'](x)
        return x
        


In [75]:
nUnitsPerLayer=512
nLayers=100
net=AI(1509,nLayers)
net=net.to(device)


In [76]:
target=data['AAPL_High']
target=target[1:]
target=torch.tensor(target.values)
target.to(device)

tensor([ 43.6375,  43.3675,  43.8425,  ..., 193.5000, 194.6600, 194.4000],
       dtype=torch.float64)

In [77]:
def trainTheModel(theModel,numepochs):
    lossfun=nn.SmoothL1Loss()
    optimizer=torch.optim.RMSprop(theModel.parameters(),lr=.01,momentum=.7)
    losses=torch.zeros(numepochs).to(device)
    for epochi in range(numepochs):
        print(epochi)
        yHat=theModel(data_corection)
        loss=lossfun(yHat,target)
        losses[epochi]=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(losses)
    return losses
    

In [79]:
numepochs=100
losses=trainTheModel(net,numepochs)
plt.plot(losses.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

0
tensor([2.1037e+16, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000

KeyboardInterrupt: 

In [ ]:
print(loss)

NameError: name 'loss' is not defined